# Graphics Processing Units (GPUs)

- Originally designed to perform computationally intensive graphics calculations
- GPUs as we know them today are largely derived from the NVIDIA GeForce 256 card, which was the first consumer level GPU
- Quickly, scientists and engineers recognized the similarities in many of their problems and those that the GPU was designed to be very good at. This led to what we now refer to as General Purpose GPU (GPGPU) programming.
    - Any problem which requires the application of similar operations to a lot of data can benefit from the architecture of GPUs
- Though GPU hardware is capable of accelerating computation in many fields other than just graphics, to take full advantage of GPUs a software platform that allows GPU programs to be written similarly to CPU programs. The first to solve this problem was NVIDIA with CUDA paradigm.
    - Soon after a vendor-agnostic language was created (OpenCL)
- CUDA and OpenCL turned GPUs into general purpose computing devices that can be leveraged alongside CPUs to perform complex calculations

# GPU vs. CPU

- GPUs are "external" devices that must interface with a CPU 
- GPUs cores are usually around 2 times slower than CPUs (2-2.5 GHz compared to 4-6 GHz)
- GPUs have **a lot** more cores than CPUs
    - For example, an AMD Threadripper 3990X has 64 cores (which is about as much as you will find) while an NVIDIA RTX 3090 has more than 10000 CUDA cores. 
- This means that processes that are algorithmically single-threaded or not compute intensive will generally be faster on CPUs than GPUs and vice-versa.

# GPU Terminology

- **Host** - the CPU that the GPU is connected to
- **Device** - the GPU
- **GPU Core** - processing unit of GPU
- **GPU Thread** - execution context of a GPU core
- **Kernel** - function created to be executed on the GPU